In [1]:
import pandas as pd
import ixmp
import message_ix

from message_ix.utils import make_df

%matplotlib inline

In [2]:
mp = ixmp.Platform()

In [3]:
model = 'Maldives energy model'
base = message_ix.Scenario(mp, model=model, scenario='baseline')
scen = base.clone(model, 'share_constraint', 'illustration of share-constraint formulation', keep_solution=False)
scen.check_out()

In [4]:
year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df['year_vtg'], year_df['year_act']
model_horizon = scen.set('year')
country = 'Maldives'

### Step 1: Defining a new 'share' 

In [14]:
scen.add_set?

In [5]:
shares = ['share_electricity', 'share_cooling', 'share_freshwater']
scen.add_set('shares', shares)

### Step 2: Defining electricity, cooling and freshwater technologies that make up the total

Define which technologies contribute to total electricity generation at the final energy level.

Define which technologies contribute to total electricity use at the useful energy level.

In [6]:
type_tec = 'electricity_total'
scen.add_cat('technology', type_tec, ['oil_ppl','solar_pv_ppl', 'battery_ppl'])
#scen.add_cat('technology', type_tec, 'solar_pv_ppl')
#scen.add_cat('technology', type_tec, 'battery_ppl')

type_tec = 'cooling_total'
scen.add_cat('technology', type_tec, ['ac', 'swac', 'dscd'])
#scen.add_cat('technology', type_tec, 'swac')
#scen.add_cat('technology', type_tec, 'dscd')

type_tec = 'freshwater_total'
scen.add_cat('technology', type_tec, ['ro_pl', 'dscd'])
#scen.add_cat('technology', type_tec, 'dscd')

In [7]:
scen.idx_names('map_shares_commodity_total')

['shares', 'node_share', 'node', 'type_tec', 'mode', 'commodity', 'level']

In [8]:
shares

['share_electricity', 'share_cooling', 'share_freshwater']

In [9]:
df = pd.DataFrame({'shares': ['share_electricity'],
                   'node_share': country,
                   'node': country,
                   'type_tec': type_tec,
                   'mode': 'standard',
                   'commodity': 'electricity',
                   'level': 'final',
})
scen.add_set('map_shares_commodity_total', df)

df = pd.DataFrame({'shares': ['share_cooling'],
                   'node_share': country,
                   'node': country,
                   'type_tec': type_tec,
                   'mode': 'standard',
                   'commodity': 'cooling',
                   'level': 'final',
})
scen.add_set('map_shares_commodity_total', df)

df = pd.DataFrame({'shares': ['share_freshwater'],
                   'node_share': country,
                   'node': country,
                   'type_tec': type_tec,
                   'mode': 'standard',
                   'commodity': 'freshwater',
                   'level': 'final',
})
scen.add_set('map_shares_commodity_total', df)

### Step 3: Defining technologies of share

In [10]:
scen.idx_names('map_shares_commodity_share')

['shares', 'node_share', 'node', 'type_tec', 'mode', 'commodity', 'level']

In [32]:
scen.add_cat?

In [11]:
#Electricity
type_tec = 'electricity_renewable'
scen.add_cat('technology', type_tec, ['solar_pv_ppl', 'battery_ppl'])
df = pd.DataFrame({'shares': ['share_electricity'],
                   'node_share': country,
                   'node': country,
                   'type_tec': type_tec,
                   'mode': 'standard',
                   'commodity': 'electricity',
                   'level': 'final',
})
scen.add_set('map_shares_commodity_share', df)

In [12]:
#cooling

type_tec = 'cooling'
scen.add_cat('technology', type_tec, ['ac', 'swac', 'dscd'])
df = pd.DataFrame({'shares': ['share_cooling'],
                   'node_share': country,
                   'node': country,
                   'type_tec': type_tec,
                   'mode': 'standard',
                   'commodity': 'cooling',
                   'level': 'useful',
})
scen.add_set('map_shares_commodity_share', df)

In [13]:
#freshwater

type_tec = 'freshwater_ro'
scen.add_cat('technology', type_tec, ['ro_pl', 'dscd'])
df = pd.DataFrame({'shares': ['share_freshwater'],
                   'node_share': country,
                   'node': country,
                   'type_tec': type_tec,
                   'mode': 'standard',
                   'commodity': 'freshwater',
                   'level': 'useful',
})

scen.add_set('map_shares_commodity_share', df)

### Step 4: Defining the share

In [14]:
# Adding data to the parameter

elec_share = {
    'shares': ['share_electricity'],
    'node_share': country,
    'year_act': [2040],
    'time': 'year',
    'unit': '-'
}

shares_factor = {
    'solar_pv_ppl': 0.7,
    'battery_ppl': 0.3   
}


for tec, val in shares_factor.items():
    df = make_df(elec_share, technology=tec, value=val)
    scen.add_par('share_commodity_lo', df)

In [15]:
# Adding data to the parameter

cool_share = {
    'shares': ['share_cooling'],
    'node_share': country,
    'year_act': [2040],
    'time': 'year',
    'unit': '-'
}

shares_factor = {
    'ac': 0.5,
    'swac': 0.3, 
    'dscd': 0.2,  
}


for tec, val in shares_factor.items():
    df = make_df(cool_share, technology=tec, value=val)
    scen.add_par('share_commodity_lo', df)

In [16]:
fresh_share = {
    'shares': ['share_freshwater'],
    'node_share': country,
    'year_act': [2040],
    'time': 'year',
    'unit': '-'
}

shares_factor = {
    'ro_pl': 0.3,
    'dscd': 0.7   
}


for tec, val in shares_factor.items():
    df = make_df(fresh_share, technology=tec, value=val)
    scen.add_par('share_commodity_lo', df)

In [17]:
scen.commit(comment='define parameters for minimum renewable share constraint')
scen.set_as_default()

In [18]:
scen.solve()

In [19]:
scen.var('OBJ')['lvl']

1542000128.0

In [20]:
from ixmp.reporting import configure
from message_ix.reporting import Reporter
import os
import matplotlib.pyplot as plt
configure(units={'replace': {'-': 'MWa'}})

<IPython.core.display.Javascript object>

In [21]:
rep = Reporter.from_scenario(scen)
plotyrs = [x for x in set(scen.set('year')) if x >= scen.firstmodelyear]

In [22]:
rep.set_filters(c = 'electricity')
elec = rep.full_key('out')
elec = elec.drop('yv','m','nd','hd','h')
elec_gen = rep.get(elec)
elec_gen

nl        t             ya    c            l    
Maldives  battery_ppl   2025  electricity  final         0.000000
                        2030  electricity  final         0.000000
                        2035  electricity  final         0.000000
                        2040  electricity  final         0.000000
          oil_ppl       2025  electricity  final    325509.419355
                        2030  electricity  final    325509.419355
                        2035  electricity  final    325509.419355
                        2040  electricity  final    325509.419355
          solar_pv_ppl  2025  electricity  final    168000.000000
                        2030  electricity  final    213150.000000
                        2035  electricity  final    235200.000000
                        2040  electricity  final    242550.000000
dtype: float64

In [23]:
rep.set_filters(c = 'cooling')
cool = rep.full_key('out')
cool = cool.drop('yv','m','nd','hd','h')
cool_gen = rep.get(cool)
cool_gen

nl        t     ya    c        l     
Maldives  ac    2025  cooling  useful    444150.0
                2030  cooling  useful    484785.0
                2035  cooling  useful    504630.0
                2040  cooling  useful    511245.0
          dscd  2025  cooling  useful         0.0
                2030  cooling  useful         0.0
                2035  cooling  useful         0.0
                2040  cooling  useful         0.0
          swac  2025  cooling  useful         0.0
                2030  cooling  useful         0.0
                2035  cooling  useful         0.0
                2040  cooling  useful         0.0
dtype: float64

In [24]:
rep.set_filters(c = 'freshwater')
wat = rep.full_key('out')
wat = elec.drop('yv','m','nd','hd','h')
wat_gen = rep.get(wat)
wat_gen

nl        t      ya    c           l     
Maldives  dscd   2025  freshwater  useful           0.0
                 2030  freshwater  useful           0.0
                 2035  freshwater  useful           0.0
                 2040  freshwater  useful           0.0
          ro_pl  2025  freshwater  useful    29200000.0
                 2030  freshwater  useful    29200000.0
                 2035  freshwater  useful    29200000.0
                 2040  freshwater  useful    29200000.0
dtype: float64

In [25]:
mp.close_db()